# Overview

This feature is a this-or-that mini game intended to teach the user about how food choices impact blood glucose levels. The user is presented with two food options and are tasked with selecting one or the other and watching how their glycemic index changes based on their choices.

# User Story

As a player, the user wants to be able to see reflections of their real life within the game through the everyday choices they make in food and how it impacts their glucose levels. This game helps simulate those choices in a controlled environment so the user can better see how their health is impacted by their choices. 

- User selects a food from two options
- The backend provides estimated glucose impact for each food via glycemic index value
- The user is responsible for ensuring their glycemic index stays low

# Features

## Frontend

The game opens with a dismissable screen explaining glycemic index, the game, and its purpose. It uses "cards" in Dexcom Green with the name, an image, and glycemic index of each food as well as a total glycemix index counter at the bottom, with a default of 0. On hover, the cards tilt left or right, and on click, the glycemic index count increases and the two food choices disappear, getting replaced with the next set of foods.

Glycemic index values are taken from online databases [here](https://glycemicindex.com/gi-search) and [here](https://glycemic-index.net/glycemic-index-chart/).

### HTML

In [ ]:
%%html
<div class="container">

    <h3 style="text-align: center;">Make the best choices for your body to keep your glucose levels low!</h3>
    
    <button class="help-btn toggle-help-btn">Help</button>
    
    <div class="help" id="help">
        <!--Instructions here-->
        <button class="help-btn toggle-help-btn">Let's Go!</button>
    </div>
    
    <div class="card-container" id="card-container"></div>
    
    <h3 style="text-align: center;">Total Glycemic Index: <span id="total-gi">0</span></h3>
    </div>

### CSS

In [ ]:
%%html
<style>

.container {
    position: relative;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
}

/*By default, the help box is shown*/
.overlay {
    position: fixed;
    top: 0;
    left: 0;
    width: 100vw;
    height: 100vh;
    background-color: #66D7D1;
    z-index: 9999;
    display: flex;
    justify-content: center;
    align-items: center;
}
.help-box {
    max-width: 900px;
    padding: 30px;
    background: transparent;
    color: #000;
    text-align: center;
}
.help {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    background-color: #66D7D1;
    z-index: 10;
    padding: 20px 40px;
    box-sizing: border-box;
    border: 2px solid transparent;
    border-radius: 10px;
    display: block;
}

.card-container {
    display: flex;
    justify-content: center;
    gap: 20px;
    margin-top: 20px;
}
/*Design for the food cards*/
.food-card {
    width: 300px;
    height: 300px;
    border: 2px solid transparent;
    border-radius: 10px;
    background-color: #58A618;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: flex-start;
    transition: transform 0.3s ease;
    padding: 10px;
    cursor: pointer;
}
.food-card img {
    display: block;
    width: 200px;
    height: 200px;
    justify-content: center;
}
.food-card div {
    display: flex;
    flex-direction: column;
    align-items: center;
    text-align: center;
    margin-top: 10px;
}

/* Tilt on hover*/
.food-card:nth-child(odd):hover {
  transform: rotate(-2deg);
}
.food-card:nth-child(even):hover {
  transform: rotate(2deg);
}
.food-card:first-child {
    margin-right: 300px;
}
</style>

### JS

In [ ]:
import { pythonURI, fetchOptions } from '{{ site.baseurl }}/assets/js/api/config.js';

// turn the help box on and off with a button press
function toggleHelp() {
    const helpBox = document.getElementById("help");
    if (helpBox.style.display === 'none') {
        helpBox.style.display = 'block';
    } else {
        helpBox.style.display = 'none';
    }
}

document.querySelectorAll('.toggle-help-btn').forEach(btn => {
    btn.addEventListener('click', toggleHelp);
});

// game logic
let totalGI = 0;
let currentPairNumber = 1;

async function fetchFoodPair(pairNumber) {
    let response = await fetch(`${pythonURI}/api/foodchoice?number=${pairNumber}`);
    let data = await response.json();
    if (data.length === 0) {
        document.getElementById("card-container").innerHTML = "<p style='text-align:center;'>Good job making healthy choices!</p>";
        return;
    }
    displayFoodPair(data);
}

function displayFoodPair(pair) {
    let container = document.getElementById("card-container");
    container.innerHTML = "";

    pair.forEach(food => {
        let foodCard = document.createElement("div");
        foodCard.classList.add("food-card");
        foodCard.setAttribute("data-glycemic", food.glycemic_index);
        let imgSrc = food.image ? `data:image/png;base64,${food.image}` : 'default-image.jpg';

        foodCard.innerHTML = `
            <img src="${imgSrc}" alt="${food.food}">
            <div>
                <span style="color: black">${food.food}</span>
                <span style="color: black">Glycemic Index: ${food.glycemic_index}</span>
            </div>
        `;

        foodCard.onclick = () => {
            totalGI += food.glycemic_index;
            document.getElementById("total-gi").textContent = totalGI;
            currentPairNumber++;
            fetchFoodPair(currentPairNumber);
        };

        container.appendChild(foodCard);
    });
}

fetchFoodPair(currentPairNumber);

### Backend

I have created both a model and an API for my feature and stored the different foods there. The foods are paired and not random, so the same 2 foods are shown at the same time every time. This ensures that the foods being compared are a fair comparison for options in a meal or snack. 

I do this by assigning each pair a number in the backend alongside food name, glycemic index, and image (stored locally to ensure fast loading times). Since images are stored locally, I also have a function that converts them to base64 so that they can be called within the backend/in python.

### Model

In [ ]:
class Food(db.Model):
    __tablename__ = 'foods'
    id = db.Column(Integer, primary_key=True)
    number = db.Column(Integer)
    food = db.Column(String, nullable=False)
    glycemic_index = db.Column(Integer, nullable=False)
    image = db.Column(String)

"""I then define all CRUD methods"""

def initFoods(): 
    food_data = [
        {
            "number": 1,
            "food": "Apple",
            "glycemic_index": 41,
            "image": "apple.png",
        },
        {
            "number": 1,
            "food": "Banana",
            "glycemic_index": 62,
            "image": "banana.png",
        },
        {
            "number": 2,
            "food": "Waffles",
            "glycemic_index": 77,
            "image": "waffle.png",
        },
        {
            "number": 2,
            "food": "Oatmeal",
            "glycemic_index": 53,
            "image": "oatmeal.png",
        }] 
"""etc"""

### API

In [ ]:
# Convert image to Base64
def encode_image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    except FileNotFoundError:
        return None

@food_api.route('/', methods=['GET'])
def get_food():
    try:
        pair_number = request.args.get('number', type=int)

        foods = Food.query.all()

        # Group foods by number
        food_dict = {}
        for food in foods:
            food_dict.setdefault(food.number, []).append(food)

        # Use the number if provided, otherwise pick first valid pair
        if pair_number:
            selected_foods = food_dict.get(pair_number, [])[:2]
        else:
            selected_foods = next((foods[:2] for foods in food_dict.values() if len(foods) >= 2), [])

        # If no foods matched  return empty
        if not selected_foods:
            return jsonify([]), 200

        food_data = [
            {
                'number': food.number,
                'food': food.food,
                'glycemic_index': food.glycemic_index,
                'image': f"/images/food/{food.image}" if food.image else None # ensure this filepath matches your filepath
            }
            for food in selected_foods
        ]

        return jsonify(food_data), 200
    except Exception as e:
        return jsonify({'error': 'Failed to fetch foods', 'message': str(e)}), 500
